In [57]:
import csv
import folium
import timeit
import numpy as np
import pandas as pd
from folium.plugins import FastMarkerCluster
from datetime import datetime
from time import process_time


In [28]:
with open('../data/Building_Permits___Current.csv',encoding='ascii',errors='ignore') as csvfile:
    permits = pd.read_csv(csvfile)
#  Change data types to reflect actual data types   
permits['Issue Date']= pd.to_datetime(permits['Issue Date'], format='%m/%d/%y')
permits['Final Date']= pd.to_datetime(permits['Final Date'], format='%m/%d/%y')
permits['Expiration Date']= pd.to_datetime(permits['Expiration Date'], format='%m/%d/%y')
permits['Application Date']= pd.to_datetime(permits['Application Date'], format='%m/%d/%y')

# Change permit value from type Object to Type float, so we can choose permits above a specified value
permits['Value'] = permits['Value'].map(lambda x: x.lstrip('$'))
permits['Value'] =permits['Value'].str.replace(',', '').astype(float)

In [70]:
core_permits = permits.filter(['Application/Permit Number', 'Address','Description', 'Category','Action Type','Value','Application Date','Issue Date','Final Date','Expiration Date','Status','Latitude','Longitude','Location'])
core_permits = core_permits[core_permits['Final Date'].notnull()]
core_permits = core_permits[core_permits['Action Type'] == 'NEW']
core_permits = core_permits[core_permits['Category'] != 'SINGLE FAMILY / DUPLEX']
core_permits = core_permits[core_permits['Value'] > 1000000]

length = core_permits.shape[0]
length


432

In [81]:
# long_permits = permits[str(permits['Final Date'] != 'NaT')] 
commercial_permits = core_permits[core_permits['Category'] == 'COMMERCIAL']
length = commercial_permits.shape[0]
commercial_permits['Accident_Count']= np.zeros(length)+1

C:\Users\Alyssa\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [121]:
np.random.randint(low = 1, high = 6, size = length)

array([5, 1, 4, 5, 2, 4, 3, 4, 4, 4, 3, 5, 4, 5, 1, 2, 2, 1, 4, 1, 3, 3, 1,
       2, 4, 4, 3, 3, 5, 1, 5, 4, 3, 5, 3, 2, 3, 3, 1, 3, 5, 2, 5, 3, 4, 4,
       4, 5, 1, 2, 1, 3, 4, 4, 3, 2, 1, 3, 3, 2, 5, 1, 2, 4, 4, 5, 3, 4, 1,
       3, 5, 5, 3, 4, 5, 3, 4, 4, 5, 5, 1, 1, 3, 2, 5, 2, 1, 1, 4, 1, 2, 2,
       5, 5, 5, 3, 4, 3, 5, 5, 2, 3, 1, 3, 1, 4, 4, 2, 1, 5, 2, 4, 3, 3, 4,
       5, 1, 2, 2, 4, 3, 2, 2, 5, 4, 2, 4, 4, 3, 4, 5, 4, 2, 1, 2, 4, 1, 2,
       1, 2, 2, 3, 5, 5, 5, 2, 1, 5, 3, 1, 3, 4, 4, 2, 3, 1, 5, 1, 5, 3, 5,
       1, 1, 5, 4, 4, 2, 1, 4, 2, 1, 4, 5, 3, 2, 2, 3, 5, 5])

In [131]:
def create_map(data):
    my_map = folium.Map(location=[np.mean(data['Latitude']), np.mean(data['Longitude'])], zoom_start = 11)
    for index, row in data.iterrows():
        folium.CircleMarker(
            location = [row['Latitude'], row['Longitude']],
            radius=row['Accident_Count'],
            fill = True,
            popup = folium.Popup(str("Address: " + row['Address'])),
            fill_color='#132b5e'
        ).add_to(my_map)
    return my_map
    
create_map(commercial_permits)

In [130]:
t = process_time()





In [122]:
elapsed_time = process_time() - t
elapsed_time

2.484375

In [126]:
commercial_permits['Accident_Count']= np.random.randint(low = 1, high = 11, size = length)

C:\Users\Alyssa\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [127]:
create_map(commercial_permits)


In [63]:
%%time

lats = core_permits['Latitude']
lons = core_permits['Longitude']

m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=11
)

FastMarkerCluster(data=list(zip(lats, lons))).add_to(m)

folium.LayerControl().add_to(m)
m

Wall time: 59.2 ms


In [64]:
m